# Background on Taproot

## Where does "taproot" come from?

Taproot is a series of bitcoin BIPS (bitcoin improvement proposals) that were merged into bitcoin-core in 2021.

It was most recent bitcoin soft-fork and it went live in November 2021.

The BIPS can be found on github. They are as follows:

- BIP340, Schnorr Signatures https://github.com/bitcoin/bips/blob/master/bip-0340.mediawiki
- BIP341, Taproot Segwit v1 https://github.com/bitcoin/bips/blob/master/bip-0341.mediawiki
- BIP342, Validating Taproot Spends https://github.com/bitcoin/bips/blob/master/bip-0342.mediawiki
- BIP350, Bech32m https://github.com/bitcoin/bips/blob/master/bip-0350.mediawiki

## What did "taproot" change?

Taproot added and modified many things in bitcoin. Including

- A new signature algorithm (Schnorr)
- A new version of Script opcodes (Segwit v1)
- A new bitcoin address checksum (bech32m)
- Removed some opcodes we know and love (OP_CHECKMULTISIG)
- Added a new opcode (OP_CHECKSIGADD)
- Added a new scripthash algorithm

# Finding Taproot in the Wild

Let's go to mempool.space and find some taproot addresses in the wild!



## How to spot a taproot address

The bech32 address has a `p` immediately following the `1`

Find a taproot address on mempool dot space.

What's the "Type" for this address?


Copy paste the "ScriptPubKey (HEX)" for the address you found into the pyblock below.

In [ ]:
scriptpubkey = "51203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953"

In [ ]:
print(scriptpubkey)

In [ ]:
len(scriptpubkey) // 2

This is a series of opcodes. If you pass this to `bitcoin-cli decodescript` what does it print back as the asm?

In [ ]:
!bitcoin-cli -regtest decodescript 51203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953

In [ ]:
output = {
  "asm": "1 3905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953",
  "desc": "addr(bcrt1p8yzasx690dpw340trtjjsryrca8mm5s29eqpcqrhhnj5z7gfe9fsysf0jl)#95u636fj",
  "address": "bcrt1p8yzasx690dpw340trtjjsryrca8mm5s29eqpcqrhhnj5z7gfe9fsysf0jl",
  "type": "witness_v1_taproot"
}

In [ ]:
print(output['asm'])

In [ ]:
output['asm'].split(" ")

In [ ]:
len(output['asm'].split(" ")[1]) // 2

In [ ]:
print(output['type'])

In [ ]:
x_bytes = bytes.fromhex(output['asm'].split(' ')[1])
print('hex:\t\t', x_bytes.hex())

x_val = int.from_bytes(x_bytes, 'big')
print('int val:\t', x_val)

## Finding the secp256k1 pubkey from a P2TR address

Now that we have the x-only pubkey, let's figure out what point that is on the secp256k1 curve.

We can do that using `lift_x_pubkey`. This takes an integer and returns a point.

In my case, I'll use `coincurve`, a python library for expressing + doing math with public keys.

In [ ]:
p = 115792089237316195423570985008687907853269984665640564039457584007908834671663
def lift_x(x_value):
    assert x_value < p
    # find y squared (y2 = x^3 + 7)
    y2 = (x_value ** 3 + 7) % p
    # find the square root of y
    y = pow(y2, (p+1)//4, p)
    assert (y ** 2 % p) == y2
    # if y is odd, find even y
    if y % 2 != 0:
        y = p - y
    return coincurve.PublicKey.from_point(x_value, y)

In [ ]:
import coincurve

point = lift_x(x_val)
print(point.format().hex())
print(point.point())

## Let's Get Familiar With The Tools!

In [ ]:
from codes import parse_tx_bytes

tx = "020000000001013c29211e2e6b9a594f28fd9e8b650dd2f64e7024d398fbc4c98f3b8800cef1350000000000ffffffff0358020000000000002251203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c95358020000000000002251203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953eba60d00000000002251203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c9530140bc93e2b4ae198c6326737b3e1525715109b6fa65cac52be9c1889408ba2d3ddeac9af8c944513905275280c9e8ac5fb6c83a7100137173c3e6a402857cb4c6de00000000"

First things first, let's update the `parse_tx_bytes` method to be able to handle segwit (any version) transactions.

In [ ]:
def parse_witness_bytes(data):
    count, size = parse_compact_size(data)
    ptr = size
    witnesses = []
    for _ in range(0, count):
        witlen, size = parse_compact_size(data[ptr:])
        ptr += size
        witnesses.append(data[ptr:ptr+witlen])
        ptr += witlen

    return witnesses, ptr

In [ ]:
from codes import parse_compact_size, parse_input_bytes, parse_output_bytes

def parse_tx_bytes_mine(tx_hex):
  tx_bytes = bytes.fromhex(tx_hex)
  tx = {}
  ptr = 0
  tx['version'] = tx_bytes[0:4]
  ptr += 4

  if tx_bytes[ptr] == 0x00:
    assert tx_bytes[ptr+1] == 0x01
    tx['marker_flag'] = bytes([0x00, 0x01])
    ptr += 2

  count, size = parse_compact_size(tx_bytes[ptr:])
  ptr += size 
  tx['inputs'] = []
  for _ in range(0, count):
    inputx, size = parse_input_bytes(tx_bytes[ptr:])
    ptr += size
    tx['inputs'].append(inputx)
 
  count, size = parse_compact_size(tx_bytes[ptr:])
  ptr += size
  tx['outputs'] = []
  for _ in range(0, count):
    outputx, size = parse_output_bytes(tx_bytes[ptr:])
    ptr += size
    tx['outputs'].append(outputx)

  if 'marker_flag' in tx:
    tx['witnesses'] = []
    for _ in range(0, len(tx['inputs'])):
        witness, size = parse_witness_bytes(tx_bytes[ptr:])
        ptr += size
        tx['witnesses'].append(witness)

  tx['locktime'] = tx_bytes[ptr:]
  return tx

In [ ]:
tx_bytes = bytes.fromhex(tx)
ptr = 4
print(tx_bytes.hex())
print(tx_bytes[ptr:].hex())

In [ ]:
pprint(parse_tx_bytes_mine(tx))

In [ ]:
txdata = parse_tx_bytes_mine(tx)

In [ ]:
assert len(txdata['witnesses'][0]) == 1

In [ ]:
print(txdata['witnesses'][0][0].hex())

In [ ]:
sig = txdata['witnesses'][0][0]

In [ ]:
print(sig.hex())

In [ ]:
print(len(sig))

## Validating a Signature

What message did this signature sign?

Finding the "sighash" of that message. 

Take the bitcoin transaction that the signature was embedded into. From this transaction, we'll create a hash.

That hash of the bitcoin transaction "message digest" which is the hash that the signature "signed".

In [ ]:
print(tx)

In [ ]:
pprint(txdata)

## Find the SigMsg for a transaction

Here we use the BIP341 Signature Validation Rules (https://github.com/bitcoin/bips/blob/master/bip-0341.mediawiki#user-content-Common_signature_message) Common signature message definition to calculate the SIGHASH_DEFAULT for a given transaction.

We'll use this to compute the sighash for this input, which we need to validate the signature.

In [ ]:
from hashlib import sha256
from codes import size_compact_size

def sigmsg_default(tx_hex, input_index, annex_bytes, amounts_bytes, scriptpubkeys_bytes, ext_flag):
    txdata = parse_tx_bytes_mine(tx_hex)
    
    result = b''
    result += bytes([0x00]) # sighash_flag
    result += txdata['version']
    result += txdata['locktime']

    all_input_outpoints = b''
    for inp in txdata['inputs']:
        # txid || vout
        all_input_outpoints += inp['txid'] + inp['vout']    
    sha_prevouts = sha256(all_input_outpoints).digest()
    result += sha_prevouts
    
    sha_amounts = sha256(b''.join(amounts_bytes)).digest()        
    result += sha_amounts
    
    spks = b''
    for spk in scriptpubkeys_bytes:
        spks += size_compact_size(len(spk)) + spk
    sha_scriptpubkeys = sha256(spks).digest()
    result += sha_scriptpubkeys
    
    all_sequences = [ i['sequence'] for i in txdata['inputs'] ]
    sha_sequences = sha256(b''.join(all_sequences)).digest()
    result += sha_sequences
    
    all_outputs = b''
    for o in txdata['outputs']:
        # amount || compact_size(scriptpubkey) || scriptpubkey
        all_outputs += o['amount']
        all_outputs += size_compact_size(len(o['scriptPubKey']))
        all_outputs += o['scriptPubKey']
    sha_outputs = sha256(all_outputs).digest()
    result += sha_outputs
    
    # data about this input
    annex_present = 1 if annex_bytes else 0
    spend_type = 2 * ext_flag + annex_present
    result += (spend_type).to_bytes(1, 'little')
    result += (input_index).to_bytes(4, 'little')
    
    if (annex_bytes):
        size_bytes = size_compact_size(len(annex_bytes))
        sha_annex = sha256(size_bytes + annex_bytes).digest()
        result += sha_annex
    
    # data about this output
    # noop for SIGHASH_DEFAULT
    
    assert len(result) <= 206
    return result

In [ ]:
spent_from_tx = parse_tx_bytes_mine("02000000000101d79ca7455775b3fc7bf65eae536faa22e6248b863247cee30765fabb4fafadb80100000000ffffffff02dcab0e00000000002251203905d81b457b42e8d5eb1ae5280c83c74fbdd20a2e401c0077bce5417909c953944be70400000000225120dc2e293b7717ece7898fefc8fd7f6da13e8fb56efac0bc3bd75183304da6ad1701409dab8ddd9c703d1acb6736bc5260b23e64999cb130b9b3ca6c2e69aa084073302b66f0ecb5b4a191618951c022da2c429ac3638613c43bbb88a761634c2cb85300000000")

print(spent_from_tx['outputs'][0])

In [ ]:
amounts_bytes = [spent_from_tx['outputs'][0]['amount']]
print("amounts:", [x.hex() for x in amounts_bytes])

scriptpubkeys_bytes = [spent_from_tx['outputs'][0]['scriptPubKey']]
print("scriptpubkeys:", [x.hex() for x in scriptpubkeys_bytes], "\n")

ext_flag = 0
annex_bytes = None
input_index = 0

sigmsg = sigmsg_default(tx, input_index, annex_bytes, amounts_bytes, scriptpubkeys_bytes, ext_flag)

print("sigmsg:", sigmsg.hex())

## Computing the Sighash

Now that we have the sighash message, we can calculate the actual "sighash" for the signature.

The `sighash` for Taproot key path spending signature valdiation is as follows:

```
sighash = tag_hash(b'TapSighash', bytes([0x00]) + sigmsg)
```

We'll need to implement the `tag_hash` function, then pass our `sigmsg` into this to find the `sighash`.

In [ ]:
from hashlib import sha256

def tag_hash(tag, data_bytes):
    taghash = sha256(tag).digest()
    return sha256(taghash + taghash + data_bytes).digest()

In [ ]:
sighash = tag_hash(b'TapSighash', bytes([0x00]) + sigmsg)
print(sighash.hex())

## Verifying the Sighash

Now that we've got the `sighash` we're finally ready to verify the signature.

For this, we'll need the `verify` function, which is defined in BIP340.

In [ ]:
# there are three inputs to the verify function: pubkey, message, signature
pk = x_val
m = sighash
sig = sig

print(x_val)

In [ ]:
from codes import n

def verify(x_val, m, sig):
    P = lift_x(x_val)
    assert len(sig) == 64
    r = int.from_bytes(sig[:32], 'big')
    if r >= p:
        return False
    s = int.from_bytes(sig[32:], 'big')
    if s >= n:
        return False
    
    P_bytes = (x_val).to_bytes(32, 'big')
    challenge_hash = tag_hash(b'BIP0340/challenge', sig[:32] + P_bytes + m)
    e = int.from_bytes(challenge_hash, 'big') % n
    
    # e * P
    eP = P.multiply(e.to_bytes(32, 'big'))
    # s * G
    sG = coincurve.PrivateKey.from_int(s).public_key
    neg_eP = invert_point(eP)
    
    # R = sG - eP = sG + -eP
    R = coincurve.PublicKey.combine_keys([sG, neg_eP])
    
    if not has_even_y(R):
        return False
    
    return R.point()[0] == r

In [ ]:
def invert_point(pt):
    x, y = pt.point()
    yneg = p - y
    return coincurve.PublicKey.from_point(x, yneg)

In [ ]:
def has_even_y(pt):
    return pt.point()[1] % 2 == 0

In [ ]:
sig = txdata['witnesses'][0][0]
verify(x_val, m, sig)

## Signing a message

Now that we've been able to successfully verify a message, let's finish implementing the second method for Schnorr: the Sign method.

This takes a private key and a message and returns a valid signature.

```
sig = Sign(privkey, msg)
```

In [ ]:
def scalar_to_pubkey(scalar_val):
    assert scalar_val != 0 and scalar_val < n
    P = coincurve.PrivateKey.from_int(scalar_val).public_key
    result = scalar_val if has_even_y(P) else n - scalar_val
    
    return result, P


def sign_msg(privkey_int, msg, nonce_int):
    d, P = scalar_to_pubkey(privkey_int)
    k, R = scalar_to_pubkey(nonce_int)
    
    bytes_R = (R.point()[0].to_bytes(32, 'big'))
    bytes_P = (P.point()[0].to_bytes(32, 'big'))
    challenge_hash = tag_hash(b'BIP0340/challenge', bytes_R + bytes_P + msg)
    e = int.from_bytes(challenge_hash, 'big') % n
    
    # ~~~BEGIN SCHNORR PART~~~
    s = (k + e * d) % n
    # ~~~ END SCHNORR PART ~~~
    
    sig = bytes_R + s.to_bytes(32, 'big')
    assert verify(P.point()[0], m, sig)
    return sig

In [ ]:
privkey = 10
nonce = 5
sig = sign_msg(privkey, m, nonce)

print(sig.hex())

## Taproot: Exploring the Depths

We've just done a Taproot keypath spend verification (~P2PK)

Now we're going to look at how to build out scripts for taproot!

We're going to make our first taproot script. (~P2WSH)

What are the steps we need to do for this:

- figure out what scripts can spend our bitcoin
- make a tree out of them (just like christmas!)
- build a merkle root
- pick a internal pubkey
- make a tweak from the merkle root
- add the tweak to the internal pubkey to get our external pubkey
- lock money up to external pubkey

### Step 1: Figure out what scripts can spend our bitcoin

- Everyone in class should make a script!
- Each individually lock up coins to the script (single script spend)
- Send all the scripts to Lisa, and then we'll build a tree out of them

#### Writing a taproot script

nifty's script: OP_4 OP_ADD OP_8 OP_EQUAL  (requires OP_4 to unlock)

Convert to script: 
        OP_4 OP_ADD OP_8 OP_EQUAL
        54935887
        
        

In [ ]:
nifty = '54935887'
casey =  'AA201ccf6c5e6212f524600fb6b20275cd6ae26dc6b812cd1f84dd3ff7d86b1937d187'
jose = '54935887'
dpp = '930200088763a820150faa5b485225f681b179f710cb169b92b401f954392eb30e677624135233f08768'
mk = '76769393010987'
damian = 'a820528f99cd13f2d438556c6f6a803458a963519d2419a40dd685597c0919d81f588754935587'
chris = '53935987'
nate = 'a82041ef4bb0b23661e66301aac36066912dac037827b4ae63a7b1165a5aa93ed4eb87'

In [ ]:
scripts = [('nifty', nifty), ('casey', casey), ('jose', jose), ('dpp', dpp), ('mk', mk), ('damian', damian), ('chris', chris), ('nate', nate)]

### Step 2: Make a tree out of your scripts

For this first round, we're just going to use a single script!

Every tree has leaves! Every script in our tree is a leaf.

Our tree is just [nifty]

In [ ]:
tree = [nifty]

In [ ]:
## Step 3: Build a Merkle root

In [ ]:
### Step 3a: Write a function to compute a leaf (hash)

In [ ]:
def make_leaf(script_bytes):
    leaf_version = 0xc0
    # leaf version + len script + script
    data = bytes([leaf_version]) + size_compact_size(len(script_bytes)) + script_bytes
    return tag_hash(b'TapLeaf', data)

In [ ]:
tree_bytes = [ bytes.fromhex(x) for x in tree ]

In [ ]:
leaf = make_leaf(tree_bytes[0])
print(leaf.hex())

In [ ]:
for name, script in [ (a,bytes.fromhex(x)) for a,x in scripts]:
    int_leaf = make_leaf(script)
    print(name, int_leaf.hex())

In this first example, where we only have one script, the "merkle root" is the leaf of the single script.

Our root = leaf.

In [ ]:
root = leaf
print(leaf.hex())

### Step 4: Pick an internal pubkey

Considerations to make when picking an *internal* pubkey:

- Does it need to be spendable?
- If yes, who has the keys to it?
- If no, how can I prove it's unspendable?


In our case, our key needs to be unspendable. We need to pick a point that we don't know the private key to.

The spec (BIP341) recommends using the following algorithm (or something like it) to pick a point where the private key is unknowns.

> One example of such a point is H = lift_x(0x50929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac0) which is constructed by taking the hash of the standard uncompressed encoding of the secp256k1 base point G as X coordinate. In order to avoid leaking the information that key path spending is not possible it is recommended to pick a fresh integer r in the range 0...n-1 uniformly at random and use H + rG as internal key. It is possible to prove that this internal key does not have a known discrete logarithm with respect to G by revealing r to a verifier who can then reconstruct how the internal key was created.

In [ ]:
# We define the nums_point as
# H(G) => sha256(uncompressed)
def nums_point(r=0):
    Gx, Gy = coincurve.PrivateKey.from_int(1).public_key.point()
    data = bytes([0x04]) + Gx.to_bytes(32, 'big') + Gy.to_bytes(32, 'big')
    h = sha256(data).digest()
    h_int = int.from_bytes(h, 'big')
    H = lift_x(h_int)
    if r > 0:
        assert r < n
        rG = coincurve.PrivateKey.from_int(r).public_key
        return coincurve.PublicKey.combine_keys([rG, H])
    return H

In [ ]:
internal_pubkey = nums_point()
print(internal_pubkey.format().hex())

pubkey_bytes = internal_pubkey.point()[0].to_bytes(32, 'big')

### Step 5: Make a tweak from the Merkle Root

What is a tweak? Ideally, a tweak is really just a number (scalar).

We're gonna take the root and a pubkey, make a hash out of both of them and then convert this hash to a number.

We'll then use that number (scalar) as if it were a private key and find the curve point at that value.

In [ ]:
def make_tweak_pubkey(pubkey, merkle_root_bytes):
    # Find the bytes for the x-value of the internal pubkey
    pubkey_bytes = pubkey.point()[0].to_bytes(32, 'big')
    
    # Compute the tweak hash (TapTweak tag + pubkey + merkle root)
    tweak_bytes = tag_hash(b'TapTweak', pubkey_bytes + merkle_root_bytes)
    # Convert that hash into a scalar (so we can make it a point)
    tweak_int = int.from_bytes(tweak_bytes, 'big')
    # Make sure our tweak is inside the "point range (n)"
    assert tweak_int < n
    
    # Multiply the tweak by the Generator Point (G) which gives us T
    T = coincurve.PrivateKey.from_int(tweak_int).public_key
    
    return T

In [ ]:
T = make_tweak_pubkey(internal_pubkey, root)
print(T.format().hex())

In [ ]:
for name, script in [ (a,bytes.fromhex(x)) for a,x in scripts]:
    root = make_leaf(script)
    T = make_tweak_pubkey(internal_pubkey, root)
    print(name, T.format().hex())

Now we have our Tweak as a point. All we need to do is combine it with the internal pubkey to get an external pubkey.

### Step 6: Add Internal Pubkey to Tweak Pubkey

For some reason the spec makes everything more fancy that it needs to be.

The biggest difference is that we "promote" the internal pubkey to a x-only pubkey (even) before we add it to the tweak.

In [ ]:
def make_external_pubkey(pubkey, T):    
    P = lift_x(pubkey.point()[0])
    Q = coincurve.PublicKey.combine_keys([P, T])
    return Q

In [ ]:
nifty

nifty_leaf = make_leaf(bytes.fromhex(nifty))
print(internal_pubkey.format().hex())
print('nifty root:', nifty_leaf.hex())
nifty_T = make_tweak_pubkey(internal_pubkey, nifty_leaf)
print('nifty tweak', nifty_T.format().hex())
nifty_Q = make_external_pubkey(internal_pubkey, nifty_T)
print('nifty!', nifty_Q.format().hex())

In [ ]:
for name, script in [ (a, bytes.fromhex(x)) for a,x in scripts]:
    root = make_leaf(script)
    T = make_tweak_pubkey(internal_pubkey, root)
    Q = make_external_pubkey(internal_pubkey, T)
    print(name, Q.format().hex())

### Step 7: Lock bitcoin up to your external pubkey! (but first make a P2TR script)

We'll take Q, drop the parity byte at the front and make this into a P2TR script.

In [ ]:
def make_p2tr(Q):
    Qx_bytes = Q.point()[0].to_bytes(32, 'big')
    witness_version = 0x51
    return bytes([witness_version]) + size_compact_size(len(Qx_bytes)) + Qx_bytes


print(make_p2tr(nifty_Q).hex())
nifty_script = make_p2tr(nifty_Q).hex()

In [ ]:
# FIXME: add `jq` to nix-shell
address_arr = !bitcoin-cli -regtest decodescript "$nifty_script" | jq -r .address
print(address_arr[0])
address = address_arr[0]

In [ ]:
!bitcoin-cli -regtest createwallet base58

In [ ]:
!bitcoin-cli -regtest -generate 101

In [ ]:
!bitcoin-cli -regtest sendtoaddress "$address" 1.0

In [ ]:
!bitcoin-cli -regtest getrawtransaction d57032ce2ac09efc013ed6786d508e8912e3486d4fddab5e4982fcdb161155a4 true

## Spending a Tapscript!

We should have a transaction output with bitcoin in it that's locked to your custom written script.

Now we want to create a transaction that spends that output.

My output is in transaction d57032ce2ac09efc013ed6786d508e8912e3486d4fddab5e4982fcdb161155a4 and the index is 1

In other words the outpoint for my bitcoins is `d57032ce2ac09efc013ed6786d508e8912e3486d4fddab5e4982fcdb161155a4:1`

Let's a build a transaction that spends this!

In [ ]:
txid = 'd57032ce2ac09efc013ed6786d508e8912e3486d4fddab5e4982fcdb161155a4'
reverse_txid = bytes.fromhex(txid)[::-1].hex()
print(reverse_txid)

amount = (1 * 10 ** 8 - 400).to_bytes(8, 'little')
print(amount.hex())

In [ ]:
!bitcoin-cli -regtest getnewaddress "" bech32m

In [ ]:
!bitcoin-cli -regtest getaddressinfo bcrt1pywat5mnqc4pyvajq3hdmujza9dwzl3lefq26w6sdhtxfgk4wyyqqsvx823 | jq -r .scriptPubKey

version: 02000000
marker+flag: 0001
inputs: 01
    txid: a4551116dbfc82495eabdd4f6d48e312898e506d78d63e01fc9ec02ace3270d5
    vout: 01000000
    scriptSig: 00
    sequence: ffffffff
outputs: 01
    amounts: 70dff50500000000
    scriptPubKey: 22 512023baba6e60c5424676408ddbbe485d2b5c2fc7f94815a76a0dbacc945aae2100
witnesses:
    ........ unsolved problem ........
locktime: 00000000

In [ ]:
tx = "0200000001a4551116dbfc82495eabdd4f6d48e312898e506d78d63e01fc9ec02ace3270d50100000000ffffffff0170dff5050000000022512023baba6e60c5424676408ddbbe485d2b5c2fc7f94815a76a0dbacc945aae210000000000"
!bitcoin-cli -regtest decoderawtransaction "$tx"

### What's in the witness stack for a leaf spend?

We need to put into the witness stack:
 - the unlocking script
 - original leaf script
 - "control block": this has all the data you need to verify the external public key
 
 #### Nifty's Unlock Script is...
 
 Well my original leaf script

In [ ]:
!bitcoin-cli -regtest decodescript "$nifty" | jq .asm

Q: How do I unlock this? Great question.

Short A: 04
Long A: The number "4". In script, we'd use the opcode 'OP_4'
but in witnesses we can't use opcodes, so we just do a "push of the byte 04".

In [ ]:
!bitcoin-cli -regtest decodescript 0104

- Unlocking script: 04
- Original leaf data??: c0 + len(nifty) + 54935887 (variable: nifty)
- Control block

#### How do we figure out a control block?


A control block has three parts.

- version byte
- the x-only pubkey of the internal key
- proof of inclusion

##### Version Byte

We're going to need Q to figure this out.

The base value for the version byte is `c0`.
With the following modification: if Q is odd, add 1 to the base value.

If Q is even (02): 0xc0

If Q is odd (03):  0xc1

In [ ]:
def control_block_version_byte(Q):
    val = 0xc0
    if Q.point()[1] % 2 != 0:
        val += 1
    return bytes([val])

print(control_block_version_byte(nifty_Q).hex())

##### The x-only pubkey of the internal key

This should just be the internal pubkey, 32 bytes.

In [ ]:
print(internal_pubkey.format()[1:].hex())

###### Proof of Inclusion

The proof of inclusion for a leaf-only spend is empty.

In [ ]:
control_block = control_block_version_byte(nifty_Q) + internal_pubkey.format()[1:]

print('witness data!')
print('03')
print('01 04')
print('04 ' + nifty)
print('21 ' + control_block.hex())

version: 02000000
marker+flag: 0001
inputs: 01
    txid: a4551116dbfc82495eabdd4f6d48e312898e506d78d63e01fc9ec02ace3270d5
    vout: 01000000
    scriptSig: 00
    sequence: ffffffff
outputs: 01
    amounts: 70dff50500000000
    scriptPubKey: 22 512023baba6e60c5424676408ddbbe485d2b5c2fc7f94815a76a0dbacc945aae2100
witnesses:
03
    01 04
    04 54935887
    21 c050929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac0
locktime: 00000000

In [ ]:
spend_tx = '02000000000101a4551116dbfc82495eabdd4f6d48e312898e506d78d63e01fc9ec02ace3270d50100000000ffffffff0170dff5050000000022512023baba6e60c5424676408ddbbe485d2b5c2fc7f94815a76a0dbacc945aae2100030104045493588721c050929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac000000000'
#!bitcoin-cli -regtest decoderawtransaction "$spend_tx"

In [ ]:
spend_tx

In [ ]:
!bitcoin-cli -regtest testmempoolaccept '["02000000000101a4551116dbfc82495eabdd4f6d48e312898e506d78d63e01fc9ec02ace3270d50100000000ffffffff0170dff5050000000022512023baba6e60c5424676408ddbbe485d2b5c2fc7f94815a76a0dbacc945aae2100030104045493588721c050929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac000000000"]'

### Let's build a taproot with mulitple leaves

Let's make a taproot with a bunch of leaves!

We have a set of scripts that we want to allow any one of them to spend this bitcoin.

Build a tree of the scripts, and then compute the merkle root.

In [ ]:
scripts

In [ ]:
balanced_tree = [[[nifty, jose], [casey, dpp]], [[mk, damian], [chris, nate]]]
unbalanced_tree = [ nifty, [ chris, [ jose, [ casey, [ dpp, [ mk, [ damian , nate ]]]]]]]

In [ ]:
balanced_tree

In [ ]:
unbalanced_tree

### Now that our tree(s) are defined, we need to calculate the merkle root

We need a function that given our tree, will return a merkle root.

In [ ]:
def make_leaf(script_bytes):
    leaf_version = 0xc0
    # leaf version + len script + script
    data = bytes([leaf_version]) + size_compact_size(len(script_bytes)) + script_bytes
    return tag_hash(b'TapLeaf', data)

In [ ]:
def taptree_builder(tree):
    if isinstance(tree, str):
        script_bytes = bytes.fromhex(tree)
        leaf_hash = make_leaf(script_bytes)
        print("leaf:", leaf_hash.hex())
        return leaf_hash
    
    # calculate the branch hash. 
    assert len(tree) == 2
    left = taptree_builder(tree[0])
    right = taptree_builder(tree[1])
    
    # we have to order the left + right "alphabetically"
    # thank you arik :)
    if left > right:
        right, left = left, right
    
    branch_hash = tag_hash(b'TapBranch', left + right)
    print("branch:", branch_hash.hex())
    return branch_hash
    

In [ ]:
balanced_root = taptree_builder(balanced_tree)
balanced_root.hex()

In [ ]:
unbalanced_root = taptree_builder(unbalanced_tree)
unbalanced_tree

In [ ]:
internal_pubkey.format().hex()

In [ ]:
balanced_tweak_key = make_tweak_pubkey(internal_pubkey, balanced_root)
balanced_tweak_key.format().hex()

In [ ]:
unbalanced_tweak_key = make_tweak_pubkey(internal_pubkey, unbalanced_root)
unbalanced_tweak_key.format().hex()

In [ ]:
Q_balanced = make_external_pubkey(internal_pubkey, balanced_tweak_key)
Q_unbalanced = make_external_pubkey(internal_pubkey, unbalanced_tweak_key)

In [ ]:
p2tr_balanced = make_p2tr(Q_balanced).hex()
p2tr_balanced

In [ ]:
p2tr_unbalanced = make_p2tr(Q_unbalanced).hex()
p2tr_unbalanced

In [ ]:
!bitcoin-cli -regtest decodescript "$p2tr_balanced" | jq -r .address

In [ ]:
!bitcoin-cli -regtest decodescript "$p2tr_unbalanced" | jq -r .address

In [ ]:
# Balanced Tree Address Lock Up
#!bitcoin-cli -regtest sendtoaddress bcrt1p6hlf4czaqztqnzyhz02rwmscr9xwh4s3q4f9l3fd209yntqu5xvqh797tq 1.1
"3f4e1ffcb93fae8f1101b321616b1d9ca18cb909d02668bbfd76c062075fa01c"

In [ ]:
# Unbalanced Tree Address Lock Up
!bitcoin-cli -regtest sendtoaddress bcrt1pnznuz6xlejyj3ym2888rwjtfacfplusskg6sefcycu5nmypuzlxspdv3ku 1.2

In [ ]:
!bitcoin-cli -regtest getrawtransaction 821c72b0a18427be1d80a8980057291e47f502ee0687e66ca5861f499449d567 true

In [ ]:
balanced_outpoint = ("821c72b0a18427be1d80a8980057291e47f502ee0687e66ca5861f499449d567", 1)
unbalanced_outpoint = ("3e1075a2dbd8f13e67d9944b464420a67e0bcd1db476b8e50a9d50aae27c76f3", 0)

txid_balanced = bytes.fromhex(balanced_outpoint[0])[::-1].hex()
txid_unbal = bytes.fromhex(unbalanced_outpoint[0])[::-1].hex()

print(txid_balanced)
print(txid_unbal)

## Let's Spend Some Things!

In [ ]:
# amount unbalanced
(12 * 10 ** 7 - 400).to_bytes(8, 'little').hex()

In [ ]:
# amount balanced
(11 * 10 ** 7 - 400).to_bytes(8, 'little').hex()

### Balanced Spend Transaction

```
version: 02000000
marker+flag: 0001
inputs: 01
    txid: 67d54994491f86a56ce68706ee02f5471e29570098a8801dbe2784a1b0721c82
    vout: 01000000
    scriptSig: 00
    sequence: ffffffff
outputs: 01
    amounts: f0758e0600000000
    scriptPubKey: 22 512023baba6e60c5424676408ddbbe485d2b5c2fc7f94815a76a0dbacc945aae2100
witnesses:
    03
        <unlock script>
        <leaf script>
        <control block!>
locktime: 00000000
```

In [ ]:
leaf_script = bytes.fromhex(damian)
leaf_script.hex()

In [ ]:
unlock_script = b'Taproot Rocks!'
unlock_script.hex()

In [ ]:
def build_proof_of_inclusion(tree, script):
    if isinstance(tree, str):
        script_bytes = bytes.fromhex(tree)
        leaf_hash = make_leaf(script_bytes)
        return leaf_hash, [], script == tree
    
    # calculate the branch hash. 
    assert len(tree) == 2
    left_hash, left_proof, left_target = build_proof_of_inclusion(tree[0], script)
    right_hash, right_proof, right_target = build_proof_of_inclusion(tree[1], script)
    
    proof = []
    if left_target:
        left_proof.append(right_hash)
        proof = left_proof
    elif right_target:
        right_proof.append(left_hash)
        proof = right_proof
        
    # we have to order the left + right "alphabetically"
    # thank you arik :)
    if left_hash > right_hash:
        right_hash, left_hash = left_hash, right_hash
    branch_hash = tag_hash(b'TapBranch', left_hash + right_hash)
    return branch_hash, proof, left_target or right_target


_, proof_of_inclusion, _ = build_proof_of_inclusion(balanced_tree, damian)

print('proof', [ x.hex() for x in proof_of_inclusion])

In [ ]:
print(b''.join(proof_of_inclusion).hex())

control_block = control_block_version_byte(Q_balanced) + internal_pubkey.format()[1:] + b''.join(proof_of_inclusion)

In [ ]:
control_block.hex()

In [ ]:
wits = [size_compact_size(len(unlock_script)) + unlock_script, 
        size_compact_size(len(leaf_script)) + leaf_script,
        size_compact_size(len(control_block)) + control_block]

print(size_compact_size(len(wits)).hex())
for witness in wits:
    print(witness.hex())

### Balanced Spend Transaction

```
version: 02000000
marker+flag: 0001
inputs: 01
    txid: 67d54994491f86a56ce68706ee02f5471e29570098a8801dbe2784a1b0721c82
    vout: 01000000
    scriptSig: 00
    sequence: ffffffff
outputs: 01
    amounts: f0758e0600000000
    scriptPubKey: 22 512023baba6e60c5424676408ddbbe485d2b5c2fc7f94815a76a0dbacc945aae2100
witnesses:
    03
        0e546170726f6f7420526f636b7321
        27a820528f99cd13f2d438556c6f6a803458a963519d2419a40dd685597c0919d81f588754935587
        81c050929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac0fa2b5d06e6fdf4ce947e00511fa6dd92294996846ec8a3b0b80d9363f0dec93ab95df7c5373a203b605e4c4773579b51f853078ba1a5f9db35df6527566c0c6d111fbd5ec2591327c312c7db37a0722205156310354d11783777f41ceff2fe96
locktime: 00000000
```

In [ ]:
tx = '0200000000010167d54994491f86a56ce68706ee02f5471e29570098a8801dbe2784a1b0721c820100000000ffffffff01f0758e060000000022512023baba6e60c5424676408ddbbe485d2b5c2fc7f94815a76a0dbacc945aae2100030e546170726f6f7420526f636b732127a820528f99cd13f2d438556c6f6a803458a963519d2419a40dd685597c0919d81f58875493558781c050929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac0fa2b5d06e6fdf4ce947e00511fa6dd92294996846ec8a3b0b80d9363f0dec93ab95df7c5373a203b605e4c4773579b51f853078ba1a5f9db35df6527566c0c6d111fbd5ec2591327c312c7db37a0722205156310354d11783777f41ceff2fe9600000000'

In [ ]:
!bitcoin-cli -regtest decoderawtransaction "$tx"

In [ ]:
!bitcoin-cli -regtest testmempoolaccept '["0200000000010167d54994491f86a56ce68706ee02f5471e29570098a8801dbe2784a1b0721c820100000000ffffffff01f0758e060000000022512023baba6e60c5424676408ddbbe485d2b5c2fc7f94815a76a0dbacc945aae2100030e546170726f6f7420526f636b732127a820528f99cd13f2d438556c6f6a803458a963519d2419a40dd685597c0919d81f58875493558781c050929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac0fa2b5d06e6fdf4ce947e00511fa6dd92294996846ec8a3b0b80d9363f0dec93ab95df7c5373a203b605e4c4773579b51f853078ba1a5f9db35df6527566c0c6d111fbd5ec2591327c312c7db37a0722205156310354d11783777f41ceff2fe9600000000"]'

### Unbalanced Spend Transaction

```
version: 02000000
marker+flag: 0001
inputs: 01
    txid: f3767ce2aa509d0ae5b876b41dcd0b7ea62044464b94d9673ef1d8dba275103e
    vout: 00000000
    scriptSig: 00
    sequence: ffffffff
outputs: 01
    amounts: 700c270700000000
    scriptPubKey: 22 512023baba6e60c5424676408ddbbe485d2b5c2fc7f94815a76a0dbacc945aae2100
witnesses:
03
0e546170726f6f7420526f636b7321
27a820528f99cd13f2d438556c6f6a803458a963519d2419a40dd685597c0919d81f588754935587
fd01010c050929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac0002a01ee216a40f3d68710b87d7fb0654a7d540e9baa3f5ba5a06170b82ed08dfa2b5d06e6fdf4ce947e00511fa6dd92294996846ec8a3b0b80d9363f0dec93a7199c3d01803160ae132c34a5aaa9e5ba44725b0e6a483caa4de6804e77e2b01bcf0ffea8d704f70610ddffb14a2cd74cbde7b4bf116605143f43fcdac8628587ce84a5b9c296529029e07e014ffdfb9bb11d201bd84e4528d4ae46d6160ead68ddbf7add8c35b385346ea0f1c3fa93bbfd9583c75ee097fe0c03c64cceccd0a7ce84a5b9c296529029e07e014ffdfb9bb11d201bd84e4528d4ae46d6160ead6
locktime: 00000000
```

In [ ]:
leaf_script = bytes.fromhex(damian)
leaf_script.hex()

In [ ]:
unlock_script = b'Taproot Rocks!'
unlock_script.hex()

In [ ]:
_, proof_of_inclusion, _ = build_proof_of_inclusion(unbalanced_tree, damian)

print([ x.hex() for x in proof_of_inclusion])

In [ ]:
control_block = control_block_version_byte(Q_unbalanced) + internal_pubkey.format()[1:] + b''.join(proof_of_inclusion)
control_block.hex()

In [ ]:
from codes import size_compact_size


wits = [size_compact_size(len(unlock_script)) + unlock_script, 
        size_compact_size(len(leaf_script)) + leaf_script,
        size_compact_size(len(control_block)) + control_block]

print(size_compact_size(len(wits)).hex())
for witness in wits:
    print(witness.hex())

In [ ]:
unbal_tx = '02000000000101f3767ce2aa509d0ae5b876b41dcd0b7ea62044464b94d9673ef1d8dba275103e0000000000ffffffff01700c27070000000022512023baba6e60c5424676408ddbbe485d2b5c2fc7f94815a76a0dbacc945aae2100030e546170726f6f7420526f636b732127a820528f99cd13f2d438556c6f6a803458a963519d2419a40dd685597c0919d81f588754935587fd0101c050929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac0002a01ee216a40f3d68710b87d7fb0654a7d540e9baa3f5ba5a06170b82ed08dfa2b5d06e6fdf4ce947e00511fa6dd92294996846ec8a3b0b80d9363f0dec93a7199c3d01803160ae132c34a5aaa9e5ba44725b0e6a483caa4de6804e77e2b01bcf0ffea8d704f70610ddffb14a2cd74cbde7b4bf116605143f43fcdac8628587ce84a5b9c296529029e07e014ffdfb9bb11d201bd84e4528d4ae46d6160ead68ddbf7add8c35b385346ea0f1c3fa93bbfd9583c75ee097fe0c03c64cceccd0a7ce84a5b9c296529029e07e014ffdfb9bb11d201bd84e4528d4ae46d6160ead600000000'

In [ ]:
!bitcoin-cli -regtest decoderawtransaction "$unbal_tx"

In [ ]:
!bitcoin-cli -regtest testmempoolaccept '["02000000000101f3767ce2aa509d0ae5b876b41dcd0b7ea62044464b94d9673ef1d8dba275103e0000000000ffffffff01700c27070000000022512023baba6e60c5424676408ddbbe485d2b5c2fc7f94815a76a0dbacc945aae2100030e546170726f6f7420526f636b732127a820528f99cd13f2d438556c6f6a803458a963519d2419a40dd685597c0919d81f588754935587fd0101c050929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac0002a01ee216a40f3d68710b87d7fb0654a7d540e9baa3f5ba5a06170b82ed08dfa2b5d06e6fdf4ce947e00511fa6dd92294996846ec8a3b0b80d9363f0dec93a7199c3d01803160ae132c34a5aaa9e5ba44725b0e6a483caa4de6804e77e2b01bcf0ffea8d704f70610ddffb14a2cd74cbde7b4bf116605143f43fcdac8628587ce84a5b9c296529029e07e014ffdfb9bb11d201bd84e4528d4ae46d6160ead68ddbf7add8c35b385346ea0f1c3fa93bbfd9583c75ee097fe0c03c64cceccd0a7ce84a5b9c296529029e07e014ffdfb9bb11d201bd84e4528d4ae46d6160ead600000000"]'

# Multisig on Taproot

In [ ]:
privkey = 8589393459493948599858588583933824757589382718181271727004040303
assert privkey < n
pubkey = coincurve.PrivateKey.from_int(privkey).public_key
pubkey.format().hex()

In [ ]:
!pwd